In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

import numpy as np
import random

import glob

import PIL
from PIL import Image

import time

import os

# Extra imports
import io
from tensorflow.python.data.experimental import AUTOTUNE

In [ ]:
# Ensure same GPU assigned by colab, for fair comparison

!nvidia-smi -L

# 1. Gather the dataset

## 1.1 Download and Extract the data

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1STYsoP85lyKAtarMRuDyTjp89tAbIDM-

In [ ]:
!unzip -o caltech256_subset_resized_cropped256x256.zip

In [ ]:
folder_paths = sorted(glob.glob("caltech256_subset_resized_cropped256x256/data/*"))[:-1]

## 1.2 Split into training, validation and testing data

In [ ]:
shuffled_paths = [] 
shuffled_labels = []

with open("caltech256_subset_resized_cropped256x256/shuffled_labels.txt") as label_file:
    label_file_lines = label_file.readlines()
    
for line in label_file_lines:
    image_path, image_label = line.strip().split(",")
    shuffled_paths.append(image_path)
    shuffled_labels.append(int(image_label))

In [ ]:
with tf.io.TFRecordWriter("caltech_dataset.tfrecords") as writer:
    for path, label in zip(shuffled_paths, shuffled_labels):
        image = Image.open("caltech256_subset_resized_cropped256x256/data/" + path)

        bytes_buffer = io.BytesIO()
        image.convert("RGB").save(bytes_buffer, "JPEG")
        image_bytes = bytes_buffer.getvalue()

        bytes_feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
        class_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))

        example = tf.train.Example(
          features=tf.train.Features(feature={
              "image": bytes_feature,
              "class": class_feature
          })
        )

        writer.write(example.SerializeToString())

        image.close()

In [ ]:
image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

In [ ]:
def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

In [ ]:
def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, [256, 256, 3])
    return image, parsed_example["class"]

In [ ]:
def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

In [ ]:
caltech_dataset = load_and_extract_images("caltech_dataset.tfrecords")

In [ ]:
train_split = 0.6
validation_split = 0.2

num_train_images = int(len(shuffled_paths) * train_split)
num_validation_images = int(len(shuffled_paths) * validation_split)

In [ ]:
train_dataset = caltech_dataset.take(num_train_images)

In [ ]:
validation_dataset = caltech_dataset.skip(num_train_images).take(num_validation_images)

In [ ]:
test_dataset = caltech_dataset.skip(num_train_images + num_validation_images)

## 1.3. Prepare data for the model

In [ ]:
image_size = 256
crop_size = 224

In [ ]:
def _train_data_preprocess_and_augment(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.random_crop(image, size=[crop_size, crop_size, 3])
    
    return image, label

In [ ]:
train_preprocessed_augmented = train_dataset.map(_train_data_preprocess_and_augment)

In [ ]:
crop_offset = (image_size - crop_size) // 2

def _test_data_preprocess(image, label):
    image = tf.cast(image, tf.float32) #  - means
    
    center_crop = tf.image.crop_to_bounding_box(
        image, crop_offset, crop_offset, crop_size, crop_size
    )
    
    return center_crop, label

In [ ]:
validation_preprocessed = validation_dataset.map(_test_data_preprocess)

## 2. Build and Train a model for 10 epochs

In [ ]:
def build_model():
    inputs = layers.Input(shape=(crop_size, crop_size, 3))

    pretrained_resnet_model = tf.keras.applications.resnet50.ResNet50(include_top=False, input_tensor=inputs)

    pretrained_resnet_model.trainable = False

    x = layers.GlobalAveragePooling2D()(pretrained_resnet_model.output)

    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(256, activation="softmax")(x)

    model = Model(inputs, outputs, name="ResNet")

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)

    model.compile(
      optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [ ]:
model = build_model()

batch_size = 128

initial_start = time.time()

for epoch in range(10):
    epoch_start = time.time()
    
    for images, label_batch in train_preprocessed_augmented.batch(batch_size):
        model.fit(images, label_batch, epochs=1, verbose=0, batch_size=batch_size)

    print(f"Evaluating Validation Accuracy...")
    
    model.evaluate(validation_preprocessed.batch(batch_size))
    
    epoch_end = time.time()
    
    print(f"Epoch Time: {epoch_end - epoch_start}")

last_end = time.time()

print(last_end-initial_start)

print(f"Total Time: {last_end-initial_start}")

# 2. Alternative model training

In [ ]:
model = build_model()

batch_size = 128

initial_start = time.time()

model.fit(train_preprocessed_augmented.batch(batch_size), validation_data=validation_preprocessed.batch(batch_size), epochs=10)

last_end = time.time()

print(last_end-initial_start)

print(f"Total Time: {last_end-initial_start}")